# Mount drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
ROOT_DIR = "/content/drive/My Drive/"
os.chdir(os.path.join(ROOT_DIR, "my_YOLOv5_auto_polyp/"))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Initial Setup

Clone repo, install dependencies, `%cd` into `./yolov5` folder and check GPU.

In [2]:
#!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -r requirements.txt  # install dependencies
os.chdir(os.path.join(ROOT_DIR, "my_YOLOv5_auto_polyp/yolov5"))


import torch
from IPython.display import Image, clear_output  # to display images
from utils.google_utils import gdrive_download  # to download models/datasets

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.9.0+cu102 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)


# Train


Start Logging Services

In [ ]:
# Start tensorboard
#%load_ext tensorboard
#%tensorboard --logdir runs/train

In [4]:
%pip install -q wandb  
clear_output()
!wandb login e938af32c1d0d1e44c4340fe1968dd3ef5cf34b6  # use 'wandb disabled' or 'wandb enabled' to disable or enable


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Preload Data from Zip -> Speeds up the Image scanning part a **lot**

In [5]:
'''SUN 1% of Data'''
# 70 epochs sinnvoll, 50 auch ok
  # Tesla V100 Zeit: Training 15:48 für s-Modell
  # max --batch-size 64 (~ 11.7 GB) scheint nicht besser zu sein als default 16?
!cp ./data/SUN/SUN_mini_split.zip /content/
!unzip -q /content/SUN_mini_split.zip -d /content/
!rm /content/SUN_mini_split.zip

In [ ]:
'''SUN 2% of Data'''
# scheint etwas schneller als 1% mini_split zu konvergieren
# 50 epochs sinnvoll
  # Tesla V100 Zeit: Training 22:50
# !cp ./data/SUN/SUN_mini_2_prozent.zip /content/
# !unzip -q /content/SUN_mini_2_prozent.zip -d /content/
# !rm /content/SUN_mini_2_prozent.zip

In [ ]:
'''mini local data '''

# !cp ./data/test_data.zip /content/
# !unzip -q /content/test_data.zip -d /content/
# !rm /content/test_data.zip

# !cp ./data/mini_data.zip /content/
# !unzip -q /content/mini_data.zip -d /content/
# !rm /content/mini_data.zip

In [ ]:
### Upload Dataset to wandb
# !python utils/wandb_logging/log_dataset.py --data ./data/SUN/SUN_mini_split.yaml

In [ ]:
### Resume training of logged checkpoint in wandb
# !python train.py --resume wandb-artifact://joshua_friedrich/YOLOv5/okvbdxmu

In [7]:
### Resume training of logged checkpoint in wandb
# !python train.py --resume wandb-artifact://joshua_friedrich/YOLOv5/3h8x0f93

### Train on SUN_Polyp_Dataset
  # pretrained: --weights yolov5s.pt 
  # OR randomly initialized: --weights '' --cfg ./models/yolov5s.yaml

  # --save_period 50   , save every 50 epochs the weihgts, useful for long training sessions, to resume training from
  # --upload_dataset   , upload dataset to W&B or connect to already uploaded one

!python train.py --img 640 --epochs 4 --data ./data/SUN/SUN_mini_split.yaml --weights ./yolov5s.pt \
 --cache-images \
 --upload_dataset \
 --name test_ray \
 --wandb-group testing





# use for data the local-yaml to get the preloaded Data
# !python train.py --img 640 --epochs 3 --data ./data/polyp_mini_local.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt \
#  --cache-images \
#  --raytune \
#  --name test_raytune \
#  --wandb-group testing 

#!python train.py --img 640 --batch 1 --epochs 3 --data ./data/polyp_mini.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt

/usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)
YOLOv5 🚀 2021-6-14 torch 1.9.0+cu102 CUDA:0 (Tesla P100-PCIE-16GB, 16280.875MB)

Namespace(adam=False, artifact_alias='latest', batch_size=16, bbox_interval=-1, bucket='', cache_images=True, cfg='', data='./data/SUN/SUN_mini_split.yaml', device='', entity=None, epochs=4, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='test_ray', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, raytune=False, rect=False, resume=False, save_dir='runs/train/test_ray2', save_period=-1, single_cls=False, sync

## 5. Appendix

In [ ]:
# Re-clone
# %cd ..
# !rm -rf yolov5 && git clone https://github.com/ultralytics/yolov5
# %cd yolov5

In [ ]:
# Apex install
# git clone https://github.com/NVIDIA/apex && cd apex && pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" . --user && cd .. && rm -rf apex

In [ ]:
# Test GCP checkpoint on COCO val2017
# %%shell
# x=best*.pt
# gsutil cp gs://*/*/weights/$x .
# python test.py --weights $x --data ./data/coco.yaml --img 736

In [ ]:
# Test multiple models on COCO val2017
# %%shell
# for x in yolov5s yolov5m yolov5l yolov5x
# do 
#   python test.py --weights $x.pt --data ./data/coco.yaml --img 640 --conf 0.001
# done

# Detection for mov Video

In [ ]:
#!python detect.py --source ../polyp_detection.mov --weights weights/best_28.06.pt --conf 0.4